[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/Extract-Earnings-Releases.ipynb)

# Extract 8-K Earnings Releases with Python -- Free, No API Key

Use **edgartools** to extract earnings releases and financial data from SEC 8-K filings -- completely free, no API key or paid subscription required. When a public company reports quarterly earnings, they file an 8-K with the SEC containing the press release and financial tables.

**What you'll learn:**
- Find earnings 8-K filings for any company
- Detect which 8-K items are in a filing (Item 2.02 = earnings)
- Extract the press release text from EX-99.1 exhibits
- Parse income statements, balance sheets, and cash flow from earnings
- Scan for earnings releases across all companies on a given date

## Install edgartools

In [ ]:
!pip install -U edgartools

## Setup

The SEC requires all automated tools to identify themselves. Replace the email below with your own -- any valid email works.

In [ ]:
from edgar import *

# The SEC requires you to identify yourself (any email works)
set_identity("your.name@example.com")

## Get a Company's Earnings Release in 3 Lines

8-K filings with Item 2.02 contain earnings announcements. edgartools finds them and extracts the press release automatically:

In [ ]:
filing = Company("AAPL").get_filings(form="8-K")[0]
eightk = filing.obj()

print(f"Company:  {filing.company}")
print(f"Date:     {filing.filing_date}")
print(f"Items:    {eightk.items}")
print(f"Earnings: {eightk.has_earnings}")

## Understand 8-K Items

Every 8-K filing reports one or more numbered items. The key ones for financial analysis:

| Item | Meaning |
|------|----------|
| **2.02** | Results of Operations and Financial Condition (= earnings) |
| **9.01** | Financial Statements and Exhibits |
| **5.02** | Departure/Election of Directors or Officers |
| **1.01** | Entry into a Material Definitive Agreement |
| **8.01** | Other Events |

Use bracket notation to read the text of any item:

In [ ]:
# Read the Item 2.02 text -- describes the earnings announcement
print(eightk["2.02"])

## Extract the Press Release

The actual earnings press release is filed as an EX-99.1 exhibit. edgartools gives you direct access:

In [ ]:
# Get the press release from the EX-99.1 exhibit
press_release = eightk.press_releases[0]

print(f"Exhibit: {press_release.document}")
print(f"Description: {press_release.description}")
print()

# Extract the full text
text = press_release.text()
print(text[:800])

## Extract Financial Tables from Earnings

When `has_earnings` is True, edgartools parses the press release and extracts structured financial tables -- income statement, balance sheet, and cash flow:

In [ ]:
# Use NVIDIA's latest earnings for clean financial tables
nvda_filings = Company("NVDA").get_filings(form="8-K")

# Find the first earnings 8-K
for f in nvda_filings:
    ek = f.obj()
    if ek.has_earnings:
        nvda_8k = ek
        print(f"Found earnings: {f.company} | {f.filing_date}")
        break

# Income statement as DataFrame
income_df = nvda_8k.get_income_statement()
print(f"\nIncome Statement ({income_df.shape[0]} rows x {income_df.shape[1]} cols)")
income_df.head(10)

In [ ]:
# Balance sheet from the same earnings filing
balance_df = nvda_8k.get_balance_sheet()
print(f"Balance Sheet ({balance_df.shape[0]} rows x {balance_df.shape[1]} cols)")
balance_df.head(10)

## Browse All Financial Tables in an Earnings Release

Earnings press releases often contain many tables beyond the standard three. Use `.earnings.financial_tables` to see them all:

In [ ]:
# List all financial tables parsed from the press release
earnings = nvda_8k.earnings

for i, table in enumerate(earnings.financial_tables):
    title = table.title or "(untitled)"
    print(f"  [{i}] {table.statement_type.value:25s} {title:35s} {table.dataframe.shape}")

## View Filing Attachments

Every 8-K has attachments -- the primary document, press release exhibits, and data files. Browse them with `.attachments`:

In [ ]:
# See all exhibits (press release documents)
filing.attachments

## Scan for Earnings Releases on Any Date

Use `get_filings()` with a date to find all 8-K filings, then filter for those with earnings:

In [ ]:
# Get all 8-K filings from a specific date
filings = get_filings(filing_date="2026-01-29", form="8-K")
print(f"Total 8-K filings on 2026-01-29: {len(filings)}")

# Scan for earnings announcements
print("\nEarnings announcements found:")
count = 0
for f in filings.head(30):
    try:
        ek = f.obj()
        if ek.has_earnings:
            count += 1
            print(f"  {f.company:40s} {f.filing_date} | Items: {ek.items}")
    except Exception:
        pass

print(f"\nFound {count} earnings releases in first 30 filings")

## Why EdgarTools?

EdgarTools is free and open-source. Compare extracting earnings data:

**With edgartools (free, no API key):**
```python
filing = Company("AAPL").get_filings(form="8-K")[0]
eightk = filing.obj()
print(eightk.press_releases[0].text())  # Full press release
income_df = eightk.get_income_statement()   # Parsed financial table
```

**Typical paid API approach ($50+/month, API key required):**
```python
from sec_api import FullTextSearchApi
api = FullTextSearchApi(api_key="YOUR_PAID_API_KEY")
query = {"query": '"Item 2.02" AND "AAPL"', "dateRange": {"startDate": "2024-01-01"}}
results = api.get_filings(query)  # Raw JSON, no financial table parsing
```

With edgartools, earnings data is automatically detected, press releases are extracted, and financial tables are parsed into DataFrames -- no query syntax, no API key, no monthly fee.

## Quick Reference

```python
from edgar import *
set_identity("your.name@example.com")

# ── Find earnings 8-K ──
filing = Company("AAPL").get_filings(form="8-K")[0]
eightk = filing.obj()

# ── Check what's in the 8-K ──
eightk.items                    # ['Item 2.02', 'Item 9.01']
eightk.has_earnings             # True if Item 2.02 + parseable tables
eightk.has_press_release        # True if EX-99 exhibit exists
eightk["2.02"]                  # Read text of any item

# ── Extract press release ──
pr = eightk.press_releases[0]
pr.text()                       # Full text of the press release
pr.document                     # Exhibit filename

# ── Extract financial tables ──
eightk.get_income_statement()   # DataFrame
eightk.get_balance_sheet()      # DataFrame
eightk.get_cash_flow_statement()# DataFrame
eightk.earnings.financial_tables # All parsed tables

# ── Browse attachments ──
filing.attachments              # All documents in the filing
```

## What's Next

You've learned how to extract earnings releases from SEC 8-K filings with Python. Here are related tutorials:

- [Extract Revenue and Earnings from SEC Filings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/extract-revenue-earnings-python.ipynb)
- [Download and Parse 10-K Annual Reports](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/download-10k-annual-report-python.ipynb)
- [Compare Company Financials with Python](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/compare-company-financials-python.ipynb)
- [Get Today's SEC Filings with Python](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/sec-filings-today-python.ipynb)

**Resources:**
- [EdgarTools Documentation](https://edgartools.readthedocs.io/)
- [GitHub Repository](https://github.com/dgunning/edgartools)
- [PyPI Package](https://pypi.org/project/edgartools/)

---

## Support EdgarTools

If you found this tutorial helpful, here are a few ways to support the project:

- **Star the repo** -- [github.com/dgunning/edgartools](https://github.com/dgunning/edgartools) -- it helps others discover edgartools
- **Visit edgartools.io** -- [edgartools.io](https://www.edgartools.io/) -- for more tutorials, articles, and updates
- **Report issues** -- found a bug or have a feature idea? [Open an issue](https://github.com/dgunning/edgartools/issues)
- **Share this notebook** -- know someone who works with SEC data? Send them the Colab link

*edgartools is free, open-source, and community-driven. No API key or paid subscription required.*